# UofT-DSI | Production - Assignment 1
Completed by: Yi-Chen Hsiao

# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

# Load dotenv extension
%load_ext dotenv

# Load environment variables from .env file
%dotenv

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

# Load the environment variable 'PRICE_DATA'
PRICE_DATA = os.getenv("PRICE_DATA")

# find the path of all parquet files in the directory 'PRICE_DATA'
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)

In [4]:
parquet_files

['../../05_src/data/prices\\AAXN\\AAXN_2001\\part.0.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2001\\part.1.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2002\\part.0.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2002\\part.1.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2003\\part.0.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2003\\part.1.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2004\\part.0.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2004\\part.1.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2005\\part.0.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2005\\part.1.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2006\\part.0.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2006\\part.1.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2007\\part.0.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2007\\part.1.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2008\\part.0.parquet',
 '../../05_src/data/prices\\AAXN\\AAXN_2008\\part.1.parquet',
 '../../

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
# Write your code below.

# load data from the parquet file paths to a dask dataframe 'dd_px', set index as 'ticker'
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
ticker,,,,,,,,,
AAXN,2001-06-08,0.566770,0.566770,0.562570,0.562570,0.562570,49200.0,AAXN.csv,2001
AAXN,2001-06-11,0.566770,0.579080,0.566770,0.570880,0.570880,106800.0,AAXN.csv,2001
AAXN,2001-06-13,0.558000,0.571000,0.555000,0.558000,0.558000,94800.0,AAXN.csv,2001
AAXN,2001-06-19,0.508333,0.508333,0.460000,0.460000,0.460000,237600.0,AAXN.csv,2001
AAXN,2001-06-20,0.458333,0.458333,0.445833,0.445833,0.445833,56400.0,AAXN.csv,2001


In [6]:
# Define a function to: (1) sort data by 'ticker' and 'Date'; (2) group by 'ticker'; 
#                       (3) add lagged values for Close and Adj_Close columns

def add_lags(dd):
    return dd.sort_values(['ticker','Date']).groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1), 
        Adj_Close_lag_1 = x['Adj Close'].shift(1))
)

# apply the'add_laggs' function using Dask's .map_partitions method,
# to ensure cosistent output across partitions  
dd_shift = dd_px.map_partitions(add_lags)

dd_shift.head(100)

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1
ticker,,,,,,,,,,,
AAXN,2001-06-08,0.566770,0.566770,0.562570,0.562570,0.562570,49200.0,AAXN.csv,2001,NaN,NaN
AAXN,2001-06-11,0.566770,0.579080,0.566770,0.570880,0.570880,106800.0,AAXN.csv,2001,0.562570,0.562570
AAXN,2001-06-13,0.558000,0.571000,0.555000,0.558000,0.558000,94800.0,AAXN.csv,2001,0.570880,0.570880
AAXN,2001-06-19,0.508333,0.508333,0.460000,0.460000,0.460000,237600.0,AAXN.csv,2001,0.558000,0.558000
AAXN,2001-06-20,0.458333,0.458333,0.445833,0.445833,0.445833,56400.0,AAXN.csv,2001,0.460000,0.460000
...,...,...,...,...,...,...,...,...,...,...,...
AAXN,2001-11-02,0.808333,0.833333,0.804167,0.822500,0.822500,470400.0,AAXN.csv,2001,0.795833,0.795833
AAXN,2001-11-05,0.812500,0.820833,0.812500,0.820833,0.820833,50400.0,AAXN.csv,2001,0.822500,0.822500
AAXN,2001-11-06,0.808333,0.816667,0.789167,0.795833,0.795833,117600.0,AAXN.csv,2001,0.820833,0.820833


In [7]:
# Add 'returns' based on Close & 'hi_lo_range'
# Assign results to 'dd_feat'

dd_feat = dd_shift.assign(
        returns = dd_shift['Close'] / dd_shift['Close_lag_1'] - 1,
        hi_lo_range = dd_shift['High'] - dd_shift['Low'])

dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
AAXN,2001-06-08,0.566770,0.566770,0.562570,0.562570,0.562570,49200.0,AAXN.csv,2001,NaN,NaN,NaN,0.004200
AAXN,2001-06-11,0.566770,0.579080,0.566770,0.570880,0.570880,106800.0,AAXN.csv,2001,0.56257,0.56257,0.014772,0.012310
AAXN,2001-06-13,0.558000,0.571000,0.555000,0.558000,0.558000,94800.0,AAXN.csv,2001,0.57088,0.57088,-0.022562,0.016000
AAXN,2001-06-19,0.508333,0.508333,0.460000,0.460000,0.460000,237600.0,AAXN.csv,2001,0.55800,0.55800,-0.175627,0.048333
AAXN,2001-06-20,0.458333,0.458333,0.445833,0.445833,0.445833,56400.0,AAXN.csv,2001,0.46000,0.46000,-0.030797,0.012500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.060000,4.530000,3.880000,4.510000,4.510000,1668500.0,ZIXI.csv,2020,4.00000,4.00000,0.127500,0.650000
ZIXI,2020-03-27,4.490000,4.710000,4.100000,4.600000,4.600000,1146800.0,ZIXI.csv,2020,4.51000,4.51000,0.019956,0.610000
ZIXI,2020-03-30,4.830000,4.870000,4.440000,4.640000,4.640000,1212000.0,ZIXI.csv,2020,4.60000,4.60000,0.008696,0.430000


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

import time
import sys
sys.path.append(os.getenv('SRC_DIR'))
from utils.logger import get_logger  
_logs = get_logger(__name__)


# convert Dask dataframe to pandas dataframe 
df_feat = dd_feat.compute()

# =============== Moving Average of Return with Pandas ===============
# Method 1: Time_based Moving average of returns (10-days window)
# This method will take the available days in the 10-days window to calculate the moving average 
# (As long as there's at least one-day has return in the 10-days window, the moving average will be calculated, otherwise, moving average will be NaN)
# Reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html


start = time.time()

# calculate rolling average 
rolling_avg = (
    df_feat.groupby('ticker')
    .rolling('10D', on='Date')['returns']
    .mean()
    .rename('returns_moving_avg_10')
)

# meerge rolling_avg dataframe back to the df_feat dataframe
df_feat = df_feat.merge(rolling_avg, on=['ticker', 'Date'], how='left')

end = time.time()

# print out the processing time for computing moving average with Pandas dataframe
_logs.info(f'Computing moving average with Pandas dataframe spent {end - start} seconds.')


2025-09-28 02:36:58,163, 1833116438.py, 36, INFO, Computing moving average with Pandas dataframe spent 0.2430412769317627 seconds.


In [9]:
df_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_moving_avg_10
ticker,,,,,,,,,,,,,,
AAXN,2001-06-08,0.566770,0.566770,0.562570,0.562570,0.562570,49200.0,AAXN.csv,2001,NaN,NaN,NaN,0.004200,NaN
AAXN,2001-06-11,0.566770,0.579080,0.566770,0.570880,0.570880,106800.0,AAXN.csv,2001,0.56257,0.56257,0.014772,0.012310,0.014772
AAXN,2001-06-13,0.558000,0.571000,0.555000,0.558000,0.558000,94800.0,AAXN.csv,2001,0.57088,0.57088,-0.022562,0.016000,-0.003895
AAXN,2001-06-19,0.508333,0.508333,0.460000,0.460000,0.460000,237600.0,AAXN.csv,2001,0.55800,0.55800,-0.175627,0.048333,-0.061139
AAXN,2001-06-20,0.458333,0.458333,0.445833,0.445833,0.445833,56400.0,AAXN.csv,2001,0.46000,0.46000,-0.030797,0.012500,-0.053554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.060000,4.530000,3.880000,4.510000,4.510000,1668500.0,ZIXI.csv,2020,4.00000,4.00000,0.127500,0.650000,0.015782
ZIXI,2020-03-27,4.490000,4.710000,4.100000,4.600000,4.600000,1146800.0,ZIXI.csv,2020,4.51000,4.51000,0.019956,0.610000,0.027711
ZIXI,2020-03-30,4.830000,4.870000,4.440000,4.640000,4.640000,1212000.0,ZIXI.csv,2020,4.60000,4.60000,0.008696,0.430000,0.048739


In [ ]:
# =============== Moving Average of Return with Pandas ===============
# Method 2: fixed-sized Moving average using .rolling(10).mean()
# This method will take the current + past 9 observations (total 10 rows) into average, not exactly 10-day window as there are missing dates in this dataset
# If there's less than 10 observations from current date back, the moving average will be NaN

df_feat_2 = df_feat.copy()

start = time.time()
df_feat_2['returns_moving_avg_10_2'] = df_feat_2['returns'].rolling(10).mean()
end = time.time()


_logs.info(f'Computing moving average with Pandas dataframe spent {end - start} seconds.')


2025-09-28 02:37:44,074, 3846290659.py, 13, INFO, Computing moving average with Pandas dataframe spent 0.008458375930786133 seconds.


In [11]:
df_feat_2

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_moving_avg_10,returns_moving_avg_10_2
ticker,,,,,,,,,,,,,,,
AAXN,2001-06-08,0.566770,0.566770,0.562570,0.562570,0.562570,49200.0,AAXN.csv,2001,NaN,NaN,NaN,0.004200,NaN,NaN
AAXN,2001-06-11,0.566770,0.579080,0.566770,0.570880,0.570880,106800.0,AAXN.csv,2001,0.56257,0.56257,0.014772,0.012310,0.014772,NaN
AAXN,2001-06-13,0.558000,0.571000,0.555000,0.558000,0.558000,94800.0,AAXN.csv,2001,0.57088,0.57088,-0.022562,0.016000,-0.003895,NaN
AAXN,2001-06-19,0.508333,0.508333,0.460000,0.460000,0.460000,237600.0,AAXN.csv,2001,0.55800,0.55800,-0.175627,0.048333,-0.061139,NaN
AAXN,2001-06-20,0.458333,0.458333,0.445833,0.445833,0.445833,56400.0,AAXN.csv,2001,0.46000,0.46000,-0.030797,0.012500,-0.053554,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.060000,4.530000,3.880000,4.510000,4.510000,1668500.0,ZIXI.csv,2020,4.00000,4.00000,0.127500,0.650000,0.015782,-0.000807
ZIXI,2020-03-27,4.490000,4.710000,4.100000,4.600000,4.600000,1146800.0,ZIXI.csv,2020,4.51000,4.51000,0.019956,0.610000,0.027711,0.004262
ZIXI,2020-03-30,4.830000,4.870000,4.440000,4.640000,4.640000,1212000.0,ZIXI.csv,2020,4.60000,4.60000,0.008696,0.430000,0.048739,0.015491


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

> 
> In moving average or time-series related analysis, I recommend using a Pandas DataFrame whenever computing resources can accommodate the data volume. Using Dask wouldn't be better for the following reasons:
>     
> 1. **Time-based rolling support**: Pandas offers flexible time-window expressions like `10D` and supports the `on='Date'` parameter in `.rolling()` method, which are essential for calendar-aware moving averages. Dask does not support these features because of its distributed architecture, which limits users to fixed-size rolling windows based on row count. These row-based windows may not align with actual time intervals.
>         
> 2. **Performance**: In my experiments, converting the Dask DataFrame to Pandas took around 11 seconds. After that, computing `.rolling(10).mean()` took approximately 11 seconds in Dask but less than 1 second in Pandas. This difference is largely due to Pandas operating entirely in memory, while Dask introduces overhead from partitioning and task scheduling. For time-sensitive analysis, using Pandas can be more efficient overall, especially when the dataset fits comfortably in memory and the conversion cost is a one-time step.

### Supporting Experiment: Moving Average of Return with Dask dataframe

In [ ]:
# =============== Moving Average of Return with Dask ===============
# Method 2: fixed-sized Moving average using .rolling(10).mean()

start = time.time()

dd_feat['returns_moving_avg_10_2'] = dd_feat['returns'].rolling(10).mean()
dd_feat.compute()

end = time.time()

_logs.info(f'Computing moving average with Dask dataframe spent {end - start} seconds.')

2025-09-28 02:44:30,960, 1831039865.py, 11, INFO, Computing moving average with Dask dataframe spent 10.774160385131836 seconds.


In [13]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_moving_avg_10_2
ticker,,,,,,,,,,,,,,
AAXN,2001-06-08,0.566770,0.566770,0.562570,0.562570,0.562570,49200.0,AAXN.csv,2001,NaN,NaN,NaN,0.004200,NaN
AAXN,2001-06-11,0.566770,0.579080,0.566770,0.570880,0.570880,106800.0,AAXN.csv,2001,0.56257,0.56257,0.014772,0.012310,NaN
AAXN,2001-06-13,0.558000,0.571000,0.555000,0.558000,0.558000,94800.0,AAXN.csv,2001,0.57088,0.57088,-0.022562,0.016000,NaN
AAXN,2001-06-19,0.508333,0.508333,0.460000,0.460000,0.460000,237600.0,AAXN.csv,2001,0.55800,0.55800,-0.175627,0.048333,NaN
AAXN,2001-06-20,0.458333,0.458333,0.445833,0.445833,0.445833,56400.0,AAXN.csv,2001,0.46000,0.46000,-0.030797,0.012500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.060000,4.530000,3.880000,4.510000,4.510000,1668500.0,ZIXI.csv,2020,4.00000,4.00000,0.127500,0.650000,-0.000807
ZIXI,2020-03-27,4.490000,4.710000,4.100000,4.600000,4.600000,1146800.0,ZIXI.csv,2020,4.51000,4.51000,0.019956,0.610000,0.004262
ZIXI,2020-03-30,4.830000,4.870000,4.440000,4.640000,4.640000,1212000.0,ZIXI.csv,2020,4.60000,4.60000,0.008696,0.430000,0.015491


In [14]:
# =============== Moving Average of Return with Dask ===============
# Method 1: Time_based Moving average of returns (10-days window)
# !!! This method is not supported in Dask

start = time.time()

rolling_avg_dd = (
    dd_feat.groupby('ticker')
    .rolling('10D', on='Date')['returns']
    .mean()
    .rename('returns_moving_avg_10')
)

dd_feat = dd_feat.merge(rolling_avg_dd, on=['ticker', 'Date'], how='left')

end = time.time()

_logs.info(f'Computing moving average with Dask dataframe spent {end - start} seconds.')


TypeError: rolling() got an unexpected keyword argument 'on'

In [15]:
start = time.time()
df_feat = dd_feat.compute()
end = time.time()


_logs.info(f'Converting Dask Dataframe to Pandas Dataframe spent {end - start} seconds.')

2025-09-28 02:46:13,877, 4060504529.py, 6, INFO, Converting Dask Dataframe to Pandas Dataframe spent 10.66221570968628 seconds.


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.